# [作業目標]
- 使用 Day 12 剛學到的方法, 對較完整的資料生成離散化特徵
- 觀察上述離散化特徵, 對於目標值的預測有沒有幫助

# [作業重點]
- 仿照 Day 12 的語法, 將年齡資料 ('DAYS_BIRTH' 除以 365) 離散化
- 繪製上述的 "離散化標籤" 與目標值 ('TARGET') 的長條圖

In [1]:
# 載入需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 設定 data_path
dir_data = 'D:\jupyter\ML_100Day_Data\ML_Data\Part01'

### 之前做過的處理

In [2]:
# 讀取資料檔
f_app_train = os.path.join(dir_data, 'application_train.csv')
app_train = pd.read_csv(f_app_train)
app_train.shape

(307511, 122)

In [3]:
# 將只有兩種值的類別型欄位, 做 Label Encoder, 計算相關係數時讓這些欄位可以被包含在內
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# 檢查每一個 column
for col in app_train:
    if app_train[col].dtype == 'object':
        # 如果只有兩種值的類別型欄位
        if len(list(app_train[col].unique())) <= 2:
            # 就做 Label Encoder, 以加入相關係數檢查
            app_train[col] = le.fit_transform(app_train[col])            
print(app_train.shape)
app_train.head()

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,M,0,1,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,F,0,0,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,1,M,1,1,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,F,0,1,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,M,0,1,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# 受雇日數為異常值的資料, 另外設一個欄位記錄, 並將異常的日數轉成空值 (np.nan)
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

# 出生日數 (DAYS_BIRTH) 取絕對值 
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])

## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [5]:
corr = app_train.corr()["TARGET"].sort_values()

In [6]:
print(corr.head(5))
print(corr.tail(5))

EXT_SOURCE_3         -0.178919
EXT_SOURCE_2         -0.160472
EXT_SOURCE_1         -0.155317
DAYS_BIRTH           -0.078239
DAYS_EMPLOYED_ANOM   -0.045987
Name: TARGET, dtype: float64
DAYS_LAST_PHONE_CHANGE         0.055218
REGION_RATING_CLIENT           0.058899
REGION_RATING_CLIENT_W_CITY    0.060893
DAYS_EMPLOYED                  0.074958
TARGET                         1.000000
Name: TARGET, dtype: float64


In [7]:
app_train['DAYS_EMPLOYED'] = abs(app_train['DAYS_EMPLOYED'])

day_data = app_train[['TARGET','DAYS_EMPLOYED']]
print(day_data.shape)
day_data = day_data[~day_data['DAYS_EMPLOYED'].isnull()]
print(day_data.shape)

day_data['DAY_BINEND'] = pd.qcut(day_data['DAYS_EMPLOYED'],10)

day_groups = day_data.groupby('DAY_BINEND').mean()
day_groups


(307511, 2)
(252137, 2)


,TARGET,DAYS_EMPLOYED
DAY_BINEND,,
"(-0.001, 333.0]",0.110323,204.789182
"(333.0, 617.0]",0.114536,471.907106
"(617.0, 919.0]",0.110775,768.543225
"(919.0, 1250.0]",0.102614,1081.058455
"(1250.0, 1648.0]",0.096453,1445.370616
"(1648.0, 2160.0]",0.085325,1883.230168
"(2160.0, 2792.0]",0.074506,2466.914227
"(2792.0, 3675.8]",0.067638,3199.396221
"(3675.8, 5333.0]",0.058135,4416.843875


In [8]:
px = day_groups.index.tolist()

py = day_groups['TARGET'].values.tolist()

sns.barplot(px,py)
plt.xticks(rotation = 75)
plt.xlabel('Day Group')
plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Day Group')
plt.show()

NameError: name 'sns' is not defined